In [ ]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
NUM_ITER = 1000000

In [ ]:
def parse_thread_times(filename):
    """Extract thread times from a given file and return the average time."""
    thread_times = []
    time_pattern = re.compile(r'Rk \d+: Time ([\d\.]+)')

    try:
        with open(filename, 'r') as f:
            for line in f:
                match = time_pattern.search(line)
                if match:
                    thread_times.append(float(match.group(1)))
        return np.mean(thread_times)
    except FileNotFoundError as e:
        raise e

In [ ]:
def collect_data():
    """Collect and organize data from constructed filenames."""
    data = {j: [] for j in range(2, 9, 2)}  # j in {2, 4, 6, 8}

    for j in data.keys():
        for k in range(10):  # k in range(10)
            # filename = f"combined_hello_4_{j}_{k}.out"
            # filename = f"combined_hello_{j}_6_{k}.out"
            # filename = f"mpi_comp_{j}_6_{k}.out"
            filename = f"mpi_comp_4_{j}_{k}.out"
            avg_time = parse_thread_times(filename)
            data[j].append(avg_time)

    return data

In [ ]:
data = collect_data()

In [ ]:
for k, v in data.items():
    print(k, v)

In [ ]:
def compute_statistics(data):
    """Compute the median and range of average times for each j."""
    stats = {}
    for j, times in data.items():
        times.sort()
        median = np.median(times)
        # median = times[len(times) // 2 - 1]
        min_time, max_time = min(times), max(times)
        lower_error = median - min(times)
        upper_error = max(times) - median
        stats[j] = (median, lower_error, upper_error, max_time, min_time)
    return stats

In [ ]:
stats2 = compute_statistics(data)

In [ ]:
for k, v in stats.items():
    print(k, v)

In [ ]:
def plot_results():
    """Plot the median values with error bars."""
    x = sorted(stats.keys())
    y = [stats[j][-1] for j in x]
    # yerr = [[stats[j][1] for j in x], [stats[j][2] for j in x]]  # Asymmetric errors
    y2 = [stats2[j][-1] for j in x]
    # yerr2 = [[stats2[j][-1] for j in x], [stats2[j][2] for j in x]]  # Asymmetric errors

    plt.figure(figsize=(8, 5))
    # plt.errorbar(x, y, yerr=yerr, fmt='o-', capsize=5, label='MCS Barrier')
    plt.plot(x, y, 'o-', label='OpenMP-MPI Combined')
    plt.plot(x, y2, 's-', label='Pure MPI')
    # plt.errorbar(x, y2, yerr=yerr2, fmt='s-', capsize=5, label='Dissemination Barrier')
    plt.xlabel("Num. of Tasks Per Node")
    plt.ylabel("Time (1e-5 s)")
    plt.title("Combined vs Pure MPI on 4 Nodes")
    plt.legend()
    plt.grid(False)
    plt.show()

In [ ]:
plot_results()

In [ ]:
from matplotlib import cm

In [ ]:
def collect_data_combined():
    """Collect and organize data from constructed filenames."""
    data = {(i, j): [] for i in range(2, 9, 2) for j in range(2, 13, 2)}

    for i, j in data.keys():
        for k in range(10):  # k in range(10)
            filename = f"combined_hello_{i}_{j}_{k}.out"
            avg_time = parse_thread_times(filename)
            if avg_time is not None:
                data[(i, j)].append(avg_time)

    return data

In [ ]:
combined_data = collect_data_combined()

In [ ]:
def compute_medians_combined(data):
    """Compute the median of average times for each (i, j)."""
    stats = {}
    for (i, j), times in data.items():
        times.sort()
        stats[(i, j)] = (times[len(times) // 2 - 1], times[0])
    return stats

In [ ]:
medians_combined = compute_medians_combined(combined_data)

In [ ]:
def plot_3d_surface(medians):
    """Plot the 3D surface with i and j as axes and median time as z-axis."""
    i_vals = sorted(set(i for i, _ in medians.keys()))
    j_vals = sorted(set(j for _, j in medians.keys()))
    X, Y = np.meshgrid(i_vals, j_vals, indexing='ij')
    Z = np.array([[medians[(i, j)][1] for j in j_vals] for i in i_vals])

    fig = plt.figure(figsize=(14, 9))
    ax = fig.add_subplot(111, projection='3d')
    # ax.plot_surface(X, Y, Z, cmap='viridis')
    surf = ax.plot_surface(X, Y, Z, cmap=cm.inferno, edgecolor='none')
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label='Time (s)')

    ax.set_xlabel("Num. of Processes")
    ax.set_ylabel("Num. of Threads")
    ax.set_zlabel("Time (1e-5 s)")
    # ax.set_title("OpenMP-MPI Combined Barrier")
    ax.view_init(elev=30, azim=200)  # Modify azim to move left
    plt.show()

In [ ]:
plot_3d_surface(medians_combined)